In [3]:
#!pip3 install lxml beautifulsoup4 argparse PyMySql pandas numpy

In [4]:
#https://franchise.ftc.go.kr/mnu/00013/program/userRqst/list.do?searchCondition=&searchKeyword=&column=brd&selUpjong=21&selIndus=&pageUnit=300&pageIndex=4

In [5]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import threading


from pprint import PrettyPrinter
pp = PrettyPrinter(indent=4)

In [6]:
resp = requests.get(
    "https://franchise.ftc.go.kr/mnu/00013/program/userRqst/list.do",
    params = {
        "searchCondition" : "",
        "searchKeyword" : "",
        "column" : "brd",
        "selUpjong":"21",
        "selIndus" : "",
        "pageUnit" : "300",
        "pageIndex" : "4"
    }
)

In [7]:
#print(resp.content.decode())

soup = BeautifulSoup(resp.content, "lxml")

In [9]:
print(len(soup.findAll("table")))

1


In [20]:
html_table = soup.find_all("table")[0]

In [21]:
type(html_table)

bs4.element.Tag

In [22]:
type(soup)

bs4.BeautifulSoup

In [28]:
print(len(html_table.findAll("tr")))
pp.pprint(html_table.findAll("tr")[1])

301
<tr>
<td>8573</td>
<td>
<a class="authCtrl" href="javascript:void(0);" onclick="fn_view('/mnu/00013/program/userRqst/view.do?firMstSn=133332');">(주)호경에프씨</a>
</td>
<td>
<a class="authCtrl" href="javascript:void(0);" onclick="fn_view('/mnu/00013/program/userRqst/view.do?firMstSn=133332');">마이하노이</a>
</td>
<td>이용재</td>
<td>20201764</td>
<td>
					
	                    
	                    
	                    
	                
	                2020.12.14				
				</td>
<td>기타 외식</td>
</tr>


In [86]:


francise_resp = requests.get(
    "https://franchise.ftc.go.kr/mnu/00013/program/userRqst/list.do",
    params = {
        "firMstSn" : "133332"
    }
)
print(francise_resp.url)

francise_resp = requests.get(
    "https://franchise.ftc.go.kr/mnu/00013/program/userRqst/view.do?firMstSn=133332"
)
print(francise_resp.url)


https://franchise.ftc.go.kr/mnu/00013/program/userRqst/list.do?firMstSn=133332
https://franchise.ftc.go.kr/mnu/00013/program/userRqst/view.do?firMstSn=133332


In [87]:
francise_soup = BeautifulSoup(francise_resp.content.strip())

In [89]:
len(francise_soup.findAll("form"))

1

In [93]:
form_tag = francise_soup.form

box_flop = form_tag.findAll("div", "box_flop")

curr_state = box_flop[0].table
table_rows = curr_state.tbody.findAll("tr")

In [165]:
type(curr_state)

bs4.element.Tag

In [123]:
for table_row in table_rows :
    #print(len(table_row.findAll("td")))
    for table_data in table_row.findAll("td") :
        value = table_data.text.strip()
        print(value.replace("\t", ""))
    print("______________")

상호
(주)호경에프씨
영업표지
마이하노이
대표자
이용재
기타 외식
______________
______________
1999.05.06
1999.05.06
031  -  987  -  2101
031  -  987  -  2107
______________
______________
20201764
2020.12.14
2022.08.31

______________


In [168]:
from dateutil.parser import parse

def extractFromTable(table_tag) :
    table_rows = table_tag.tbody.findAll("tr")
    
    business_name, business_sign, representative, business_type = list(map(
        lambda table_data : table_data.text.strip().replace("\t", '').split("\n")[-1],
        table_rows[0].findAll("td")
    ))
    representative = representative.split(",")

    corporate_registration_date, business_registration_date, key_number, key_fax_number = list(map(
        lambda table_data : table_data.text.strip().replace(" ", "").replace("\xa0",""),
        table_rows[2].findAll("td")
    ))
    corporate_registration_date = parse(corporate_registration_date)
    business_registration_date = parse(business_registration_date)

    registration_id, initial_registration_date, final_registration_date, _ = list(map(
        lambda table_data : table_data.text.strip(),
        table_rows[4].findAll("td")
    ))
    initial_registration_date = parse(initial_registration_date)
    final_registration_date = parse(final_registration_date)

    return [
        business_name, business_sign, representative, business_type,
        corporate_registration_date, business_registration_date, key_number, key_fax_number,
        registration_id, initial_registration_date, final_registration_date
    ]

extractFromTable(curr_state)

['(주)호경에프씨',
 '마이하노이',
 ['이용재'],
 '기타 외식',
 datetime.datetime(1999, 5, 6, 0, 0),
 datetime.datetime(1999, 5, 6, 0, 0),
 '031-987-2101',
 '031-987-2107',
 '20201764',
 datetime.datetime(2020, 12, 14, 0, 0),
 datetime.datetime(2022, 8, 31, 0, 0)]

In [160]:
def extractFromTable(table_tag) :
    rows = table_tag.tbody.findAll("tr")
    
    l1 = list(map(
        lambda table_data : table_data.text.strip().replace("\t", '').split("\n")[-1],
        rows[0].findAll("td")
    ))
    l2 = list(map(
        lambda table_data : table_data.text.strip().replace(" ", "").replace("\t",""),
        rows[2].findAll("td")
    ))

    l3 = list(map(
        lambda table_data : table_data.text.strip(),
        rows[4].findAll("td")
    ))

    print(l1)
    print(l2)
    print(l3)

extractFromTable(curr_state)

['(주)호경에프씨', '마이하노이', '이용재', '기타 외식']
['1999.05.06', '1999.05.06', '031\xa0-\xa0987\xa0-\xa02101', '031\xa0-\xa0987\xa0-\xa02107']
['20201764', '2020.12.14', '2022.08.31', '']


In [101]:
for table_data in table_rows[0].findAll("td") :
    print(table_data

<label class="hidden" for="label01">상호</label>
<label class="hidden" for="label02">영업표지</label>
<label class="hidden" for="label03">대표자</label>
None
